# Data

In [3]:
from src.mtal.data_collect import get_pair_df
from src.mtal.data_collect import get_spot_pairs

cryptos = get_spot_pairs()

# df = get_pair_df(pair="BTCUSDT")
df = get_pair_df(pair="BTCUSDT", limit=500, frequency="1w")
df.tail(14)

,Open Time,Open,High,Low,Close,Volume,Close Time
338,2024-02-05,42582.88,48592.66,42258.10,48299.99,262240.483570,2024-02-11 23:59:59.999
339,2024-02-12,48300.00,52816.62,47710.01,52137.67,310862.301700,2024-02-18 23:59:59.999
340,2024-02-19,52137.68,52985.00,50521.00,51728.85,223366.161860,2024-02-25 23:59:59.999
341,2024-02-26,51728.85,64000.00,50901.44,63113.97,417907.833830,2024-03-03 23:59:59.999
342,2024-03-04,63113.97,69990.00,59005.00,68955.88,481870.181782,2024-03-10 23:59:59.999
343,2024-03-11,68955.88,73777.00,64533.00,68393.48,477496.917520,2024-03-17 23:59:59.999
344,2024-03-18,68393.47,68956.00,60775.00,67209.99,409762.744140,2024-03-24 23:59:59.999
345,2024-03-25,67210.00,71769.54,66385.06,71280.01,235409.957550,2024-03-31 23:59:59.999
346,2024-04-01,71280.00,71288.23,64493.07,69360.39,274227.114880,2024-04-07 23:59:59.999
347,2024-04-08,69360.38,72797.99,60660.57,65661.84,348008.319040,2024-04-14 23:59:59.999


# EMA backtest

In [9]:
from src.mtal.utils import generate_pinescript
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester 

short_ema = 55 
long_ema = 59 

df = get_pair_df(pair="BTCUSDT", limit=500, frequency="1d")

tester = MACrossBacktester(
    short_ma=short_ema, long_ma=long_ema, data=df
)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

print(results)


BacktestResults(pnl=2144.9217058808244, pnl_percentage=2.1449217058808245, max_drawdown=0.5371681911903082, win_rate=1.0, average_return=0.791543627253674, trade_number=2, entry_dates=[Timestamp('2023-01-06 23:59:59.999000'), Timestamp('2023-10-21 23:59:59.999000')], exit_dates=[Timestamp('2023-08-22 23:59:59.999000'), Timestamp('2024-05-08 23:59:59.999000')], entry_prices=[16950.65, 29909.8], exit_prices=[26056.0, 61193.03], profit_pct_history=[0.5371681911903082, 1.0459190633170399], profit_history=[537.1681911903082, 1607.7535146905157])


In [4]:
from itertools import combinations
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester


ema_values = range(1, 60)  

combinations = [(short_ema, long_ema) for short_ema, long_ema in combinations(ema_values, 2)]

results = {}

for short_ema, long_ema in combinations:
    tester = MACrossBacktester(short_ma=short_ema, long_ma=long_ema, data=df)
    test_results = tester.run()
    results[(short_ema, long_ema)] = test_results

best_combination = max(results, key=lambda x: results[x].pnl)  # Ajuster le critère si nécessaire
best_result = results[best_combination]

print(f"Meilleure combinaison: Short EMA = {best_combination[0]}, Long EMA = {best_combination[1]}")
print(f"Résultat du test: {best_result}")


Meilleure combinaison: Short EMA = 55, Long EMA = 59
Résultat du test: BacktestResults(pnl=2697.621393230537, pnl_percentage=1.6976213932305368, max_drawdown=0.25442455932371505, win_rate=1.0, average_return=0.7024548572963697, trade_number=2, entry_dates=[Timestamp('2023-01-19 23:59:59.999000'), Timestamp('2023-10-22 23:59:59.999000')], exit_dates=[Timestamp('2023-08-23 23:59:59.999000'), Timestamp('2024-04-25 23:59:59.999000')], entry_prices=[21071.59, 29992.46], exit_prices=[26432.72, 64498.34])


# EMA  price above

In [11]:
from src.mtal.utils import generate_pinescript
from src.mtal.backtesting.ma_cross_backtest import MACrossPriceAboveBacktester 

short_ema = 19 
long_ema = 47 

df = get_pair_df(pair="BTCUSDT", limit=500, frequency="1d")

tester = MACrossPriceAboveBacktester(
    short_ma=short_ema, long_ma=long_ema, data=df
)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

print(results)

BacktestResults(pnl=915.8910177936802, pnl_percentage=0.9158910177936802, max_drawdown=-0.02370493211961077, win_rate=0.5, average_return=0.19830610882863214, trade_number=4, entry_dates=[Timestamp('2023-01-06 23:59:59.999000'), Timestamp('2023-06-22 23:59:59.999000'), Timestamp('2023-10-05 23:59:59.999000'), Timestamp('2024-01-27 23:59:59.999000')], exit_dates=[Timestamp('2023-03-03 23:59:59.999000'), Timestamp('2023-07-24 23:59:59.999000'), Timestamp('2023-10-11 23:59:59.999000'), Timestamp('2024-04-13 23:59:59.999000')], entry_prices=[16950.65, 29884.92, 27410.39, 42120.63], exit_prices=[22354.34, 29176.5, 26875.52, 63924.51], profit_pct_history=[0.3187895449437041, -0.02370493211961077, -0.01951340349407648, 0.5176532259845118], profit_history=[318.7895449437041, -31.261816642942886, -25.12404807214443, 653.4873375650635])


In [12]:
from itertools import combinations
from src.mtal.backtesting.ma_cross_backtest import MACrossPriceAboveBacktester


ema_values = range(1, 60)  

combinations = [(short_ema, long_ema) for short_ema, long_ema in combinations(ema_values, 2)]

results = {}

for short_ema, long_ema in combinations:
    tester = MACrossPriceAboveBacktester(short_ma=short_ema, long_ma=long_ema, data=df)
    test_results = tester.run()
    results[(short_ema, long_ema)] = test_results

best_combination = max(results, key=lambda x: results[x].pnl)  # Ajuster le critère si nécessaire
best_result = results[best_combination]

print(f"Meilleure combinaison: Short EMA = {best_combination[0]}, Long EMA = {best_combination[1]}")
print(f"Résultat du test: {best_result}")


Meilleure combinaison: Short EMA = 19, Long EMA = 47
Résultat du test: BacktestResults(pnl=2607.640337455214, pnl_percentage=2.6076403374552144, max_drawdown=-0.03862851230654122, win_rate=0.75, average_return=0.40636658069417786, trade_number=4, entry_dates=[Timestamp('2022-12-21 00:00:00'), Timestamp('2023-06-23 00:00:00'), Timestamp('2023-10-06 00:00:00'), Timestamp('2024-01-28 00:00:00')], exit_dates=[Timestamp('2023-05-18 00:00:00'), Timestamp('2023-08-17 00:00:00'), Timestamp('2024-01-27 00:00:00'), Timestamp('2024-04-30 00:00:00')], entry_prices=[16896.15, 29884.92, 27410.39, 42120.63], exit_prices=[27405.62, 28730.51, 41823.51, 63866.0], profit_pct_history=[0.6220038292747162, -0.03862851230654122, 0.5258268853525981, 0.5162641204559382], profit_history=[622.0038292747162, -62.65559488039536, 819.947225271639, 1228.344877789254])


# VWMA cross 

In [7]:
from src.mtal.utils import generate_pinescript
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester 

short_ema = 56 
long_ema = 58 

df = get_pair_df(pair="BTCUSDT", limit=500, frequency="1d")

tester = MACrossBacktester(short_ma=short_ema, long_ma=long_ema, data=df, ma_type="vwma"
)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

print(results)
# print(generate_pinescript(entry_dates, exit_dates))



BacktestResults(pnl=676.7053417979312, pnl_percentage=0.6767053417979312, max_drawdown=-0.13466265754549342, win_rate=0.3333333333333333, average_return=0.12387330002909236, trade_number=6, entry_dates=[Timestamp('2023-04-16 23:59:59.999000'), Timestamp('2023-04-19 23:59:59.999000'), Timestamp('2023-05-28 23:59:59.999000'), Timestamp('2023-07-04 23:59:59.999000'), Timestamp('2023-10-13 23:59:59.999000'), Timestamp('2024-02-05 23:59:59.999000')], exit_dates=[Timestamp('2023-04-17 23:59:59.999000'), Timestamp('2023-05-27 23:59:59.999000'), Timestamp('2023-06-05 23:59:59.999000'), Timestamp('2023-08-17 23:59:59.999000'), Timestamp('2024-01-31 23:59:59.999000'), Timestamp('2024-05-03 23:59:59.999000')], entry_prices=[30304.65, 28797.1, 28065.0, 30766.51, 26862.0, 42708.7], exit_prices=[29430.27, 26854.27, 25728.2, 26623.41, 42580.0, 62882.01], profit_pct_history=[-0.028852997807267234, -0.06746616846835266, -0.08326385177267057, -0.13466265754549342, 0.5851388578661306, 0.47234661790220744

In [6]:
from itertools import combinations
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester


ema_values = range(1, 60)  

combinations = [(short_ema, long_ema) for short_ema, long_ema in combinations(ema_values, 2)]

results = {}

for short_ema, long_ema in combinations:
    tester = MACrossBacktester(short_ma=short_ema, long_ma=long_ema, data=df, ma_type="vwma")
    test_results = tester.run()
    results[(short_ema, long_ema)] = test_results

best_combination = max(results, key=lambda x: results[x].pnl)  # Ajuster le critère si nécessaire
best_result = results[best_combination]

print(f"Meilleure combinaison: Short EMA = {best_combination[0]}, Long EMA = {best_combination[1]}")
print(f"Résultat du test: {best_result}")


/home/romain/mtal/src/mtal/analysis.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[get_ma_names(span, "vwma")] = vwma
/home/romain/mtal/src/mtal/analysis.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[get_ma_names(span, "vwma")] = vwma
/home/romain/mtal/src/mtal/analysis.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

Meilleure combinaison: Short EMA = 56, Long EMA = 58
Résultat du test: BacktestResults(pnl=3577.2975760474555, pnl_percentage=2.5772975760474557, max_drawdown=-0.17333902100011148, win_rate=0.6, average_return=0.1732078101534478, trade_number=10, entry_dates=[Timestamp('2022-03-19 23:59:59.999000'), Timestamp('2022-08-12 23:59:59.999000'), Timestamp('2022-11-02 23:59:59.999000'), Timestamp('2023-01-06 23:59:59.999000'), Timestamp('2023-01-09 23:59:59.999000'), Timestamp('2023-04-17 23:59:59.999000'), Timestamp('2023-04-20 23:59:59.999000'), Timestamp('2023-07-05 23:59:59.999000'), Timestamp('2023-10-14 23:59:59.999000'), Timestamp('2024-02-06 23:59:59.999000')], exit_dates=[Timestamp('2022-04-04 23:59:59.999000'), Timestamp('2022-09-13 23:59:59.999000'), Timestamp('2022-11-05 23:59:59.999000'), Timestamp('2023-01-07 23:59:59.999000'), Timestamp('2023-04-16 23:59:59.999000'), Timestamp('2023-04-18 23:59:59.999000'), Timestamp('2023-05-28 23:59:59.999000'), Timestamp('2023-08-18 23:59:59

# VWMA above

In [12]:
from src.mtal.backtesting.ma_cross_backtest import MACrossPriceAboveBacktester 

short_ema = 2 
long_ema = 5

df = get_pair_df(pair="BTCUSDT", limit=500, frequency="1d")

tester = MACrossPriceAboveBacktester(short_ma=short_ema, long_ma=long_ema, data=df, ma_type="vwma"
)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

print(results)

BacktestResults(pnl=3108.9087794278794, pnl_percentage=3.1089087794278796, max_drawdown=-0.04734561471993561, win_rate=0.5111111111111111, average_return=0.03482758631691941, trade_number=45, entry_dates=[Timestamp('2023-01-02 23:59:59.999000'), Timestamp('2023-01-20 23:59:59.999000'), Timestamp('2023-02-01 23:59:59.999000'), Timestamp('2023-02-13 23:59:59.999000'), Timestamp('2023-03-02 23:59:59.999000'), Timestamp('2023-03-12 23:59:59.999000'), Timestamp('2023-03-26 23:59:59.999000'), Timestamp('2023-03-29 23:59:59.999000'), Timestamp('2023-04-05 23:59:59.999000'), Timestamp('2023-04-09 23:59:59.999000'), Timestamp('2023-04-25 23:59:59.999000'), Timestamp('2023-05-03 23:59:59.999000'), Timestamp('2023-05-15 23:59:59.999000'), Timestamp('2023-05-23 23:59:59.999000'), Timestamp('2023-05-27 23:59:59.999000'), Timestamp('2023-06-09 23:59:59.999000'), Timestamp('2023-06-16 23:59:59.999000'), Timestamp('2023-06-30 23:59:59.999000'), Timestamp('2023-07-08 23:59:59.999000'), Timestamp('2023-

In [4]:
from itertools import combinations
from src.mtal.backtesting.ma_cross_backtest import MACrossPriceAboveBacktester 

ema_values = range(1, 60)  
combinations = [(short_ema, long_ema) for short_ema, long_ema in combinations(ema_values, 2)]

results = {}

for short_ema, long_ema in combinations:
    tester = MACrossPriceAboveBacktester(short_ma=short_ema, long_ma=long_ema, data=df, ma_type="vwma")
    test_results = tester.run()
    results[(short_ema, long_ema)] = test_results

best_combination = max(results, key=lambda x: results[x].pnl)  # Ajuster le critère si nécessaire
best_result = results[best_combination]

print(f"Meilleure combinaison: Short EMA = {best_combination[0]}, Long EMA = {best_combination[1]}")
print(f"Résultat du test: {best_result}")

Meilleure combinaison: Short EMA = 2, Long EMA = 5
Résultat du test: BacktestResults(pnl=3115.5126566516756, pnl_percentage=3.1155126566516755, max_drawdown=-0.04734561471993561, win_rate=0.5111111111111111, average_return=0.03486297837435225, trade_number=45, entry_dates=[Timestamp('2022-12-25 23:59:59.999000'), Timestamp('2023-01-02 23:59:59.999000'), Timestamp('2023-01-20 23:59:59.999000'), Timestamp('2023-02-01 23:59:59.999000'), Timestamp('2023-02-13 23:59:59.999000'), Timestamp('2023-03-02 23:59:59.999000'), Timestamp('2023-03-12 23:59:59.999000'), Timestamp('2023-03-26 23:59:59.999000'), Timestamp('2023-03-29 23:59:59.999000'), Timestamp('2023-04-05 23:59:59.999000'), Timestamp('2023-04-09 23:59:59.999000'), Timestamp('2023-04-25 23:59:59.999000'), Timestamp('2023-05-03 23:59:59.999000'), Timestamp('2023-05-15 23:59:59.999000'), Timestamp('2023-05-23 23:59:59.999000'), Timestamp('2023-05-27 23:59:59.999000'), Timestamp('2023-06-09 23:59:59.999000'), Timestamp('2023-06-16 23:59:5

EMA price above + close en dessous de la long: 2-5 nous donne un bon x3

# HMA cross

In [14]:
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester 

short_ema = 6
long_ema = 15 

df = get_pair_df(pair="BTCUSDT", limit=500, frequency="1d")

tester = MACrossBacktester(short_ma=short_ema, long_ma=long_ema, data=df, ma_type="hma"
)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

print(results)

BacktestResults(pnl=2200.4663874061985, pnl_percentage=2.2004663874061983, max_drawdown=-0.08877992904949492, win_rate=0.4666666666666667, average_return=0.02955498590272919, trade_number=45, entry_dates=[Timestamp('2023-01-02 23:59:59.999000'), Timestamp('2023-01-21 23:59:59.999000'), Timestamp('2023-02-02 23:59:59.999000'), Timestamp('2023-02-13 23:59:59.999000'), Timestamp('2023-02-28 23:59:59.999000'), Timestamp('2023-03-07 23:59:59.999000'), Timestamp('2023-03-12 23:59:59.999000'), Timestamp('2023-03-30 23:59:59.999000'), Timestamp('2023-04-09 23:59:59.999000'), Timestamp('2023-04-24 23:59:59.999000'), Timestamp('2023-05-05 23:59:59.999000'), Timestamp('2023-05-14 23:59:59.999000'), Timestamp('2023-05-23 23:59:59.999000'), Timestamp('2023-05-27 23:59:59.999000'), Timestamp('2023-06-08 23:59:59.999000'), Timestamp('2023-06-13 23:59:59.999000'), Timestamp('2023-06-16 23:59:59.999000'), Timestamp('2023-07-03 23:59:59.999000'), Timestamp('2023-07-10 23:59:59.999000'), Timestamp('2023-

In [5]:
from itertools import combinations
from src.mtal.backtesting.ma_cross_backtest import MACrossPriceAboveBacktester 

ema_values = range(2, 60)  
combinations = [(short_ema, long_ema) for short_ema, long_ema in combinations(ema_values, 2)]

results = {}

for short_ema, long_ema in combinations:
    tester = MACrossPriceAboveBacktester(short_ma=short_ema, long_ma=long_ema, data=df, ma_type="hma")
    test_results = tester.run()
    results[(short_ema, long_ema)] = test_results

best_combination = max(results, key=lambda x: results[x].pnl)
best_result = results[best_combination]

print(f"Meilleure combinaison: Short EMA = {best_combination[0]}, Long EMA = {best_combination[1]}")
print(f"Résultat du test: {best_result}")

Meilleure combinaison: Short EMA = 6, Long EMA = 15
Résultat du test: BacktestResults(pnl=2716.3159239382962, pnl_percentage=2.7163159239382964, max_drawdown=-0.04331413447480797, win_rate=0.5121951219512195, average_return=0.035472466141545, trade_number=41, entry_dates=[Timestamp('2023-01-01 23:59:59.999000'), Timestamp('2023-01-21 23:59:59.999000'), Timestamp('2023-02-13 23:59:59.999000'), Timestamp('2023-03-07 23:59:59.999000'), Timestamp('2023-03-12 23:59:59.999000'), Timestamp('2023-03-30 23:59:59.999000'), Timestamp('2023-04-09 23:59:59.999000'), Timestamp('2023-04-24 23:59:59.999000'), Timestamp('2023-05-05 23:59:59.999000'), Timestamp('2023-05-14 23:59:59.999000'), Timestamp('2023-05-23 23:59:59.999000'), Timestamp('2023-05-27 23:59:59.999000'), Timestamp('2023-06-08 23:59:59.999000'), Timestamp('2023-06-13 23:59:59.999000'), Timestamp('2023-06-16 23:59:59.999000'), Timestamp('2023-07-03 23:59:59.999000'), Timestamp('2023-07-10 23:59:59.999000'), Timestamp('2023-07-21 23:59:59

# VZO-RSI

In [15]:
from src.mtal.backtesting.vzo_rsi import VZO_RSI_let_grey
from src.mtal.data_collect import get_pair_df
from src.mtal.data_collect import get_spot_pairs
from src.mtal.utils import generate_pinescript

cryptos = get_spot_pairs()
df = get_pair_df(pair="BTCUSDT", limit=500, frequency="1d")

tester = VZO_RSI_let_grey(data=df, grey_zone=5, span=14)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

# print(generate_pinescript(entry_dates=entry_dates, exit_dates=exit_dates))
print(results)

BacktestResults(pnl=1588.6837760966764, pnl_percentage=1.5886837760966763, max_drawdown=-0.09246567973708945, win_rate=0.625, average_return=0.14728744338945263, trade_number=8, entry_dates=[Timestamp('2023-01-11 23:59:59.999000'), Timestamp('2023-02-18 23:59:59.999000'), Timestamp('2023-03-13 23:59:59.999000'), Timestamp('2023-04-27 23:59:59.999000'), Timestamp('2023-05-28 23:59:59.999000'), Timestamp('2023-06-20 23:59:59.999000'), Timestamp('2023-09-19 23:59:59.999000'), Timestamp('2024-01-29 23:59:59.999000')], exit_dates=[Timestamp('2023-02-10 23:59:59.999000'), Timestamp('2023-03-03 23:59:59.999000'), Timestamp('2023-04-21 23:59:59.999000'), Timestamp('2023-05-08 23:59:59.999000'), Timestamp('2023-06-01 23:59:59.999000'), Timestamp('2023-07-24 23:59:59.999000'), Timestamp('2024-01-14 23:59:59.999000'), Timestamp('2024-04-13 23:59:59.999000')], entry_prices=[17943.26, 24631.95, 24113.48, 29472.77, 28065.0, 28307.99, 27210.26, 43302.7], exit_prices=[21625.19, 22354.34, 27262.84, 276

In [5]:
from src.mtal.backtesting.vzo_rsi import VZO_RSI
from src.mtal.data_collect import get_pair_df
from src.mtal.data_collect import get_spot_pairs
from src.mtal.utils import generate_pinescript

cryptos = get_spot_pairs()
df = get_pair_df(pair="BTCUSDT", limit=500, frequency="1d")

tester = VZO_RSI(data=df, grey_zone=-2, span=14)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

# print(generate_pinescript(entry_dates=entry_dates, exit_dates=exit_dates))
print(results)

BacktestResults(pnl=1899.7489908558132, pnl_percentage=1.8997489908558132, max_drawdown=-0.037859259866757256, win_rate=0.23529411764705882, average_return=0.03968659462380547, trade_number=34, entry_dates=[Timestamp('2023-01-11 23:59:59.999000'), Timestamp('2023-02-15 23:59:59.999000'), Timestamp('2023-02-17 23:59:59.999000'), Timestamp('2023-03-13 23:59:59.999000'), Timestamp('2023-04-26 23:59:59.999000'), Timestamp('2023-05-02 23:59:59.999000'), Timestamp('2023-05-28 23:59:59.999000'), Timestamp('2023-06-06 23:59:59.999000'), Timestamp('2023-06-16 23:59:59.999000'), Timestamp('2023-07-07 23:59:59.999000'), Timestamp('2023-07-10 23:59:59.999000'), Timestamp('2023-08-01 23:59:59.999000'), Timestamp('2023-08-08 23:59:59.999000'), Timestamp('2023-08-14 23:59:59.999000'), Timestamp('2023-08-29 23:59:59.999000'), Timestamp('2023-09-14 23:59:59.999000'), Timestamp('2023-09-28 23:59:59.999000'), Timestamp('2023-10-04 23:59:59.999000'), Timestamp('2023-10-06 23:59:59.999000'), Timestamp('202

In [17]:
from itertools import product
from src.mtal.backtesting.ma_cross_backtest import MACrossPriceAboveBacktester 

span_values = range(1, 40)  
grey_zone_values = range(-30, 30)
combinations = [(span, grey) for span, grey in product(span_values, grey_zone_values)]

results = {}

for span, grey in combinations:
    tester = VZO_RSI(data=df, grey_zone=grey, span=span)
    test_results = tester.run()
    results[(span, grey)] = test_results

best_combination = max(results, key=lambda x: results[x].pnl)
best_result = results[best_combination]

print(f"Meilleure combinaison: span = {best_combination[0]}, grey = {best_combination[1]}")
print(f"Résultat du test: {best_result}")

ValueError: min() arg is an empty sequence

In [ ]:
from itertools import product
from src.mtal.backtesting.ma_cross_backtest import MACrossPriceAboveBacktester 

span_values = range(1, 40)  
grey_zone_values = range(-30, 30)
combinations = [(span, grey) for span, grey in product(span_values, grey_zone_values)]

results = {}

for span, grey in combinations:
    tester = VZO_RSI_let_grey(data=df, grey_zone=grey, span=span)
    test_results = tester.run()
    results[(span, grey)] = test_results

best_combination = max(results, key=lambda x: results[x].pnl)
best_result = results[best_combination]

print(f"Meilleure combinaison: span = {best_combination[0]}, grey = {best_combination[1]}")
print(f"Résultat du test: {best_result}")

Meilleure combinaison: span = 20, grey = -20
Résultat du test: BacktestResults(pnl=2454.2186073553016, pnl_percentage=2.4542186073553016, max_drawdown=-0.035984149733906935, win_rate=0.5, average_return=0.2998845894125007, trade_number=6, entry_dates=[Timestamp('2023-01-10 23:59:59.999000'), Timestamp('2023-03-05 23:59:59.999000'), Timestamp('2023-03-12 23:59:59.999000'), Timestamp('2023-08-23 23:59:59.999000'), Timestamp('2023-08-27 23:59:59.999000'), Timestamp('2023-09-12 23:59:59.999000')], exit_dates=[Timestamp('2023-03-04 23:59:59.999000'), Timestamp('2023-03-07 23:59:59.999000'), Timestamp('2023-08-17 23:59:59.999000'), Timestamp('2023-08-25 23:59:59.999000'), Timestamp('2023-09-11 23:59:59.999000'), Timestamp('2024-05-08 23:59:59.999000')], entry_prices=[17440.66, 22430.24, 21997.11, 26432.72, 26101.77, 25840.1], exit_prices=[22346.57, 22197.96, 26623.41, 26060.01, 25162.52, 61193.03], profit_pct_history=[0.2812915336919589, -0.010355662712481117, 0.21031399124703196, -0.0141003